In [2]:
#%pip install gradio
#%pip install opencv-python

In [ ]:
"""
# 얼굴 감지
haarcascade_frontalface_default	가장 표준적인 정면 얼굴 감지.	학습 시작 시 사용자 인증/출석 체크의 기본 단계로 활용.
haarcascade_frontalface_alt	default보다 조금 더 정교한 정면 얼굴 감지.	default 실패 시 AI 면접 및 집중도 모니터링의 정확도를 높이기 위한 대안 모델.
haarcascade_frontalface_alt2	성능과 속도의 균형을 잡은 정면 얼굴 감지.	다양한 조명 조건에서의 얼굴 추적에 활용.
haarcascade_profileface	사람의 측면 얼굴 감지.	집중도 모니터링 시 학습자가 화면에서 고개를 돌리고 있는지(딴짓 여부) 파악.

# 눈 & 표정 & 시선 감지
haarcascade_eye	일반적인 눈 감지.	졸음 감지의 기초 로직(눈 깜빡임 빈도 분석).
haarcascade_eye_tree_eyeglasses	안경 착용자의 눈 감지에 특화.	안경을 쓴 학습자의 시선 처리 정확도 향상 (AI 면접, 발표 코칭).
haarcascade_lefteye_2splits	왼쪽 눈만 감지.	보다 정밀한 시선 방향 추적 (학습자가 화면의 특정 영역을 보고 있는지 분석).
haarcascade_righteye_2splits	오른쪽 눈만 감지.	보다 정밀한 시선 방향 추적.
haarcascade_smile	입꼬리의 움직임을 감지하여 웃는 표정 감지.	AI 면접 기능에서 지원자의 표정 안정성 및 미소 유지 여부 피드백.

# 신체 자세 및 행동 감지
haarcascade_upperbody	사람의 상반신 감지 (머리, 어깨).	발표 코칭 및 AI 면접 시 카메라 앵글 내 자세 안정성 체크.
haarcascade_lowerbody	사람의 하반신 감지.	학습자가 자리에서 일어섰는지 (자리 이탈) 감지하여 집중도 저하 알림.
haarcascade_fullbody	사람의 전신 감지.	발표 코칭 시 발표자의 전신 제스처 변화 분석 (활용 빈도는 낮을 수 있음).

# 기타 객체 감지 및 동물
haarcascade_frontalcatface	고양이 얼굴 감지.	활용도 낮음. (단, 펫캠 기반 학습 환경 시 활용 가능)
haarcascade_frontalcatface_extended	확장된 고양이 얼굴 감지.	활용도 낮음.
haarcascade_russian_plate_number	러시아어 차량 번호판 감지.	활용도 없음 (학습 플랫폼과 무관).
haarcascade_license_plate_rus_16stages	16단계의 러시아어 차량 번호판 감지 (더 정교함).	활용도 없음 (학습 플랫폼과 무관).
"""

In [35]:
import cv2
import gradio as gr
from PIL import Image

# cv2.data.haarcascades => 'c:\\Users\\EL32\\AppData\\Local\\Programs\\Python\\Python311\\Lib\\site-packages\\cv2\\data\\'
cascade_file = cv2.data.haarcascades + "haarcascade_frontalface_default.xml"

# Haar Cascade 분류기 객체 생성. 앞서 지정한 XML 파일에 저장된 사전 학습된 특징 데이터를 메모리에 로드하며, 이미지에서 얼굴을 감지할 준비가 완료
face_cascade = cv2.CascadeClassifier(cascade_file)

def detect_face(image_arr) :
    # read-only 배열을 writable 배열로 변환
    image_arr = image_arr.copy()

    # detectMultiScale는 이미지를 줄이고 Face를 찾는 과정을 반복한다.
    # 전달 파라미터 중, scaleFactor는 이미지를 줄이는 비율을 의미. (보통 1.1~1.3으로 적용하고, 비율 작을수록 정확하지만 연산 속도 느림
    # minNeighbors : 객체(얼굴)로 인정하기 위해 최소한 몇 번의 인접한 검출 사각형이 있어야 하는지를 지정. (이 값이 높을수록 오탐지는 줄지만, 정확한 감지 기회도 줄어들 수 있음.)
    # minSize : 감지할 객체의 최소 크기를 튜플 (width, height)로 지정. 이보다 작은 객체는 무시합니다.
    # maxSize : 감지할 객체의 최대 크기를 지정. 이보다 큰 객체는 무시합니다.
    bounding_boxes = face_cascade.detectMultiScale(
        image=image_arr,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(50, 50)
    )
    """
    # a는 detectMultiScale와 같은 좌표. b는 검출된 각 face의 신뢰도를 의미
    a, b = face_cascade.detectMultiScale2(
        image=image_arr,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(50, 50)
    )

    # a는 detectMultiScale와 같은 좌표. b는 검출된 각 face의 신뢰도를 의미. c는 가중치 값
    a, b, c = face_cascade.detectMultiScale3(
        image=image_arr,
        scaleFactor=1.1,
        minNeighbors=5,
        minSize=(50, 50),
        outputRejectLevels=True
    )
    """

    for bounding_box in bounding_boxes : 
        x, y, w, h =  bounding_box

        # 이미지에 바운딩 박스 그리기(색상은 BGR순. 마지막 3은 바운딩박스 테두리 굵기)
        cv2.rectangle(image_arr, (x, y), (x + w, y + h), (0,255,0), 3)

    # COLOR_BGR2RGB = BRG 형식을 RGB 형태로 변환
    # 웹캠에서 받아오는 이미지는 RGB 형태임. 아래 코드 실행 시 BGR로 변경되어 얼굴이 파랗게 나옴..
    #image_arr = cv2.cvtColor(image_arr, cv2.COLOR_BGR2RGB)
    return image_arr


with gr.Blocks() as demo : 
    def stream_webcam(image_arr) : 
        result_image = detect_face(image_arr)
        return result_image
    
    with gr.Row() :
        # 디바이스의 캠을 이용하여, 실시간으로 웹캠의 화면을 Image Array 형태로 받아옴.
        stream_image = gr.Image(label="실시간 웹캠", sources="webcam", streaming=True)

        # 웹캠에서 Face 감지 결과를 표출할 컴포넌트
        result_image = gr.Image(label="검출 결과", interactive=False)

    stream_image.stream(fn=stream_webcam, inputs=[stream_image], outputs=[result_image])

demo.launch()


#result_image = detect_face(image_arr = cv2.imread("files/test6.jpg"))
#pil_image = Image.fromarray(result_image)
#pil_image

* Running on local URL:  http://127.0.0.1:7873
* To create a public link, set `share=True` in `launch()`.
